In [2]:
from bs4 import BeautifulSoup

In [3]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service

In [4]:
import time

In [23]:
#You need to have driver corresponding to your Google Chrome browser or update browser
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [6]:
#CS  - CS 
#EC  - ECE
#CI  - AI
#CSE - CS+AI
RESULT_FOR="CSE"
YEAR="21"

In [7]:
usns=[]
# Change Range According to Number of students
if(RESULT_FOR!="CSE"):
    #Students USN 
    for i in range(1,366):
        usn='01FE'+YEAR+'B'+RESULT_FOR+((3-len(str(i)))*'0')+str(i)
        usns.append(usn)
    
else:
    #CS + AI USNS
    for i in range(2,367):
        usn='01FE'+YEAR+'BCS'+((3-len(str(i)))*'0')+str(i)
        usns.append(usn)
    for i in range(1,63):
        usn='01FE'+YEAR+'BCI'+((3-len(str(i)))*'0')+str(i)
        usns.append(usn)

### Run the cell below and wait until Google Chrome opens 

In [46]:
url="https://eresults.kletech.ac.in/index.php"

#This is the path name of chrome driver on my PC
service = Service()
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=service, options=options)

driver.get(url)
driver.maximize_window()

### When website Opens then change the captcha below and run the cell

### My college website has a glitch that its captcha doesn't change if we navigate back on the website

In [47]:
#put the captcha as shown in website and run this cell
captcha="ACQPG" #this was the current captcha when I was using

In [48]:
result = {}
result['NAME'] = [None] * len(usns)
result['USN'] = ['None'] * len(usns)
result['SGPA'] = [None] * len(usns)
result['CGPA'] = [None] * len(usns)

for number in range(len(usns)):
    username = usns[number]
    driver.find_element(By.NAME, "usn").send_keys(username)
    driver.find_element(By.NAME, "osolCatchaTxt").send_keys(captcha)

    # Click the Result button
    driver.find_element(By.XPATH, '//button[text()="Result"]').click()
    
    try:
        # Wait for the link to be present and clickable, then click it
        WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//a[@onclick=\"return submitfrom('" + username + "', '27');\"]"))
        ).click()
    except Exception as E:
        print('USN:',username,'RESULT IS TAL')
        driver.back()
        # Wait for the input fields to be present
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.NAME, "usn"))
        )
        driver.find_element(By.NAME, "usn").clear()  # Clearing the already filled detail
        driver.find_element(By.NAME, "osolCatchaTxt").clear() 
        continue
        
    try:    
        name = driver.find_element(By.XPATH, '//div[contains(@class,"uk-card uk-card-body stu-data stu-data1")]')
        table = driver.find_element(By.XPATH, '//table[contains(@class,"uk-table uk-table-striped res-table")]/tbody')
        sgpa = driver.find_element(By.XPATH, '//div[contains(@class,"uk-card uk-card-default uk-card-body credits-sec3")]')
        cgpa = driver.find_element(By.XPATH, '//div[contains(@class,"uk-card uk-card-default uk-card-body credits-sec4")]')
        sub = table.text.split('\n')
        grades = ""
        print()
        print('=======================')
        print('Name:', name.text)
        print('USN:', username)
        print('=======SUBJECTS========')
        for i in range(len(sub)):
            t = sub[i].split(" ")
            title = " ".join(t[1:-3])
            grades = t[-1]
            print(title, ':', grades)
            if result.get(title) is None:
                result[title] = [None] * len(usns)
            result[title][number] = grades

        print('=======SUBJECTS========')
        print('SGPA:', sgpa.text[5:])
        print('CGPA', cgpa.text[5:])
        print('=======================')
        print()
        result['NAME'][number] = name.text
        result['USN'][number] = username
        result['SGPA'][number] = sgpa.text[5:]
        result['CGPA'][number] = cgpa.text[5:]
        
        driver.back()
    except Exception as e:
        print("Error in fetching values for USN", username, ":", e)
        driver.back()
        
    driver.back()  # Navigating back by clicking back button

    # Wait for the input fields to be present
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.NAME, "usn"))
    )
    driver.find_element(By.NAME, "usn").clear()  # Clearing the already filled detail
    driver.find_element(By.NAME, "osolCatchaTxt").clear()  # Clearing the CAPTCHA field as it will be Entered again
    

print('==========')
print("Completed")
print('==========')
#print(result)


Name: AMOGH M MURAGODMATH
USN: 01FE21BCS002
=======SUBJECTS========
Semantic Web : A
Computer Networks Lab : C
Computer Networks-2 : B
Cloud Computing : A
Blockchain and Distributed Ledgers : B
Minor Project -1 : A
Minor Project -2 : S
=======SUBJECTS========
SGPA: 8.83
CGPA 8.35


Name: SAISAMARTH S UDIKERI
USN: 01FE21BCS003
=======SUBJECTS========
Semantic Web : B
Computer Networks Lab : B
Computer Networks-2 : B
Cloud Computing : B
Blockchain and Distributed Ledgers : B
Minor Project -1 : S
Minor Project -2 : S
=======SUBJECTS========
SGPA: 8.86
CGPA 6.99


Name: JAGADEESH A KALYAN SHETTI
USN: 01FE21BCS004
=======SUBJECTS========
Semantic Web : D
Computer Networks Lab : A
Computer Networks-2 : E
Cloud Computing : X
Blockchain and Distributed Ledgers : D
Minor Project -1 : A
Minor Project -2 : S
=======SUBJECTS========
SGPA: 6.79
CGPA 6.84


Name: OM BEVINAKATTI
USN: 01FE21BCS005
=======SUBJECTS========
Semantic Web : B
Computer Networks Lab : B
Computer Networks-2 : C
Cloud Computi

In [49]:
import pandas as pd
df=pd.DataFrame.from_dict(result)
df = df[df['USN'].str.contains('None') == False]
df.to_csv('result_B'+RESULT_FOR+'_'+YEAR+'_Vi.csv',index=False)